In [90]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Color
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.formatting.rule import FormulaRule
from openpyxl.formatting.rule import CellIsRule
import re

In [98]:
def apply_currency_formatting(worksheet, col_letter):
    custom_num_format = '[Green]$#,##0;[Red]"($"#,##0);$#,##0'

    # Apply the custom number format to the entire column
    for row in range(2, worksheet.max_row + 1):
        cell = worksheet[f'{col_letter}{row}']
        cell.number_format = custom_num_format

In [81]:
def format_excel(writer, df, sheet_name):
    workbook = writer.book
    worksheet = workbook[sheet_name]
                
    # Find the columns by header names
    campain_link = None
    days_before = None
    days_after = None    
    roas_3_days_before = None    
    roas_after = None
    roas_3_days_change = None
    profit_before = None   
    profit_after = None
    
    for col in worksheet.iter_cols(1, worksheet.max_column):
        if col[0].value == 'CAMPAIGN Link':
            campain_link = col[0].column_letter
        elif col[0].value == 'Days Before':
            days_before = col[0].column_letter
        elif col[0].value == 'Days After':
            days_after = col[0].column_letter
        elif col[0].value == 'ROAS 3 Days Before':
            roas_3_days_before = col[0].column_letter        
        elif col[0].value == 'ROAS After':
            roas_after = col[0].column_letter            
        elif col[0].value == 'ROAS 3 Days Change':
            roas_3_days_change = col[0].column_letter         
        elif col[0].value == 'Profit Before':
            profit_before = col[0].column_letter     
        elif col[0].value == 'Profit After':
            profit_after = col[0].column_letter             
               
    # Auto-adjust columns' width
    for column in worksheet.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column: 

            # Special handling for 'CAMPAIGN Link' column
            if column_letter == campain_link:
                max_length = 48
            else: 
                cell_value = cell.value
                if len(str(cell_value)) > max_length:
                    max_length = len(str(cell_value))
                    max_length = max_length

            adjusted_width = max_length + 2
            worksheet.column_dimensions[column_letter].width = adjusted_width
        
    # Format 'CAMPAIGN Link' as a hyperlink
    if 'CAMPAIGN Link' in df.columns:
        campaign_link_index = df.columns.get_loc('CAMPAIGN Link') + 1
        for row in range(2, worksheet.max_row + 1):
            cell = worksheet.cell(row=row, column=campaign_link_index )
            if cell.value and isinstance(cell.value, str) and "HYPERLINK" in cell.value:
                link = re.search(r'HYPERLINK\("(.*)",.*\)', cell.value)
                if link:
                    cell.hyperlink = link.group(1)
                    cell.value = extract_display_text_from_hyperlink(cell.value)
                    cell.style = 'Hyperlink'
                    cell.font = Font(color='0000FF', underline='single')       

    # Round columns
    columns_two_decimals = [roas_3_days_before, roas_after, roas_3_days_change]
    
    for col_letter in columns_two_decimals:
        for row in range(2, worksheet.max_row + 1):  
            cell = f'{col_letter}{row}'
            worksheet[cell].number_format = '0.00'   
           
    # Currency formatting for 'Profit Before' and 'Profit After'
    if profit_before:
        apply_currency_formatting(worksheet, profit_before)
    if profit_after:
        apply_currency_formatting(worksheet, profit_after)

    # Formatting for 'ROAS 3 Days Change'
    if roas_3_days_change :
        for cell in worksheet[roas_3_days_change ]:
            cell.font = Font(bold=True)
        green_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
        red_text = Font(color='00FF0000')
        green_text = Font(color='008E40')
        worksheet.conditional_formatting.add(f'{roas_3_days_change }2:{roas_3_days_change }{worksheet.max_row}',
                                             CellIsRule(operator='lessThan', formula=['0'], font=red_text))
        worksheet.conditional_formatting.add(f'{roas_3_days_change }2:{roas_3_days_change }{worksheet.max_row}',
                                             CellIsRule(operator='greaterThan', formula=['0'], font=green_text))

    # Green fill for positive 'Profit After'
    if profit_after:
        green_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
        worksheet.conditional_formatting.add(f'{profit_after}2:{profit_after}{worksheet.max_row}',
                                             CellIsRule(operator='greaterThan', formula=['0'], fill=green_fill))
        

    # Conditional Formatting
    if profit_after and days_after:
        formula = f'AND(${profit_after}2<0, ${days_after}2>6)'
        yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

        worksheet.conditional_formatting.add(f'A2:{profit_after}{worksheet.max_row}',
                                             FormulaRule(formula=[formula], stopIfTrue=True, fill=yellow_fill))
        

In [10]:
def csv_to_formatted_excel(csv_file, excel_file, sheet_name='Sheet1'):
    # Read CSV file
    df = pd.read_csv(csv_file)

    # Create a Pandas Excel writer using Openpyxl as the engine
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        # Convert the dataframe to an XlsxWriter Excel object
        df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Format the Excel file
        format_excel(writer, df, sheet_name)

In [100]:
csv_file = 'df.csv'
excel_file = 'excel_output.xlsx'
csv_to_formatted_excel(csv_file, excel_file)